In [46]:
import os

In [47]:
os.chdir("C:\\Users\\user\\Desktop\\BHS\\Coding\\3_ML_Ops_Pipeline_AWS")

In [48]:
%pwd

'C:\\Users\\user\\Desktop\\BHS\\Coding\\3_ML_Ops_Pipeline_AWS'

In [49]:
# src\cnnClassifier\entity\config_entity.py
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [50]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [51]:
# src\cnnClassifier\config\configuration.py

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [52]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [53]:
# src\cnnClassifier\components\prepare_base_model.py

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        """ VGG16 Base Model
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        """
        #MobileNetV2
        self.model = tf.keras.applications.MobileNetV2(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        # Input Layer 
        input_layer = tf.keras.layers.Input(shape=model.input_shape[1:])
        
        # Rescaling-Layer hinzufügen
        rescaling_layer = tf.keras.layers.Rescaling(1./127.5, offset=-1)(input_layer)

        # Base Model mit Rescaling verbinden
        base_output = model(rescaling_layer)

        # Add GlobalAveragePooling2D
        global_pooling = tf.keras.layers.GlobalAveragePooling2D()(base_output)

        # Add Dropout
        dropout = tf.keras.layers.Dropout(0.2)(global_pooling)  # Dropout rate von 0.2

        # Dense Layer dynamisch basierend auf der Anzahl der Klassen
        if classes == 1:
            # Binary Classification
            activation = 'sigmoid'
            loss = tf.keras.losses.BinaryCrossentropy()
            metrics = [tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="accuracy")]
        else:
            # Multi-Class Classification
            activation = 'softmax'
            loss = tf.keras.losses.CategoricalCrossentropy()
            metrics = ["accuracy"]

        # Finaler Layer (Dense)
        prediction = tf.keras.layers.Dense(
            units=classes,  # Anzahl der Klassen
            activation=activation  # Dynamische Auswahl der Aktivierungsfunktion
        )(dropout)

        # Zusammenfassung finales Modell
        full_model = tf.keras.models.Model(
            inputs=input_layer,
            outputs=prediction
        )

        # Kompilieren Modell
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=loss,
            metrics=metrics
        )

        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        self.full_model.summary()
    

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)




In [54]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-11-27 09:16:27,343: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-27 09:16:27,346: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-27 09:16:27,348: INFO: common: created directory at: artifacts]
[2024-11-27 09:16:27,351: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-11-27 09:16:28,379: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 160, 160, 3)]     0         
                                                                 
 rescaling_4 (Rescaling)     (None, 160, 160, 3)       0         
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 128